In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import vgg16, VGG16_Weights
from PIL import Image
from tqdm import tqdm

In [2]:
# -------------------------------------------------------------
# Define Device
# -------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------------------------------------
# Load Pretrained VGG16 with 1024-D Output
# -------------------------------------------------------------
vgg = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

# Freeze base feature extractor (optional)
for param in vgg.features.parameters():
    param.requires_grad = False

# Custom model to reduce features from 25088 → 1024
vgg_model = nn.Sequential(
    vgg.features,
    nn.AdaptiveAvgPool2d((7, 7)),
    nn.Flatten(),
    nn.Linear(512 * 7 * 7, 1024),
    nn.ReLU()
)

vgg_model.eval().to(device)

# -------------------------------------------------------------
# Define Image Transformation
# -------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:
# -------------------------------------------------------------
# Feature Extraction Function
# -------------------------------------------------------------
def extract_features(image_path, transform, device=device):
    """Extract VGG16-1024 features from a single image."""
    try:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        # Extract features using VGG16
        with torch.no_grad():
            features = vgg_model(image).squeeze().cpu().numpy()

        return features
    except Exception as e:
        print(f"Skipping frame: {image_path} due to error: {e}")
        return None

In [9]:
# -------------------------------------------------------------
# Paths to RGB and Label CSV
# -------------------------------------------------------------
rgb_path = "D:/Datasets/Datasets/GTEA/GTEA_NEW/S2_Cheese_C1"
label_csv_path = "D:/Datasets/Datasets/GTEA/GTEA_NEW/Label_CSV/S2_Cheese_C1.csv"
output_csv = "Feature_VGG16Only/Feature_S2_Cheese_C1_GTEA.csv"

# Load label CSV
labels_df = pd.read_csv(label_csv_path)

In [10]:
# -------------------------------------------------------------
# Extract Features and Save to CSV
# -------------------------------------------------------------
S = 1 # Sample every 5th frame
features_list = []

# Get all RGB frames (sampled)
all_frames = sorted(os.listdir(rgb_path))[::S]

for frame in tqdm(all_frames, desc="Extracting RGB Features"):
    rgb_frame_path = os.path.join(rgb_path, frame)

    # Extract VGG16 features
    rgb_features = extract_features(rgb_frame_path, transform, device)

    if rgb_features is not None:
        frame_number = int(frame.split('_')[-1].split('.')[0])

        # Get corresponding label
        label_row = labels_df[(labels_df['StartFrame'] <= frame_number) & (labels_df['EndFrame'] >= frame_number)]

        action_label = label_row.iloc[0]['ActionLabel'] if not label_row.empty else 0

        # Save [FrameName, ActionLabel, FeatureVector]
        features_list.append([frame, action_label] + rgb_features.tolist())

# -------------------------------------------------------------
# Save Features to CSV
# -------------------------------------------------------------
if len(features_list) == 0:
    raise ValueError("No valid features extracted. Please check the dataset paths and feature extraction function.")

# Define column names
columns = ["Frame", "ActionLabel"] + [f"Feature_{i}" for i in range(1024)]
df = pd.DataFrame(features_list, columns=columns)
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
df.to_csv(output_csv, index=False)

print(f"✅ Feature extraction completed! Saved to: {output_csv}")

Extracting RGB Features: 100%|███████████████████████████████████████████████████████| 634/634 [00:07<00:00, 83.36it/s]


✅ Feature extraction completed! Saved to: Feature_VGG16Only/Feature_S2_Cheese_C1_GTEA.csv


In [8]:
import pandas as pd
df=pd.read_csv("Feature_VGG16Only/Feature_0005_Meccano.csv")
df

,Frame,ActionLabel,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,...,Feature_1014,Feature_1015,Feature_1016,Feature_1017,Feature_1018,Feature_1019,Feature_1020,Feature_1021,Feature_1022,Feature_1023
0,frame_0000.jpg,0,0.066825,0.104420,0.000000,0.000000,0.000000,0.00000,0.803856,0.885755,...,0.0,0.000000,0.0,0.050389,0.964257,0.000000,0.029447,0.000000,0.0,0.162589
1,frame_0010.jpg,0,0.000000,0.000000,0.000000,1.243672,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.0,0.568067,0.000000,0.317512,0.000000,0.421041,0.0,0.000000
2,frame_0020.jpg,0,0.000000,0.000000,0.000000,1.146604,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.0,0.429557,0.000000,0.725178,0.000000,0.381150,0.0,0.000000
3,frame_0030.jpg,0,0.000000,0.000000,0.053867,1.135250,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.0,0.046750,0.000000,1.193105,0.392156,0.000000,0.0,0.000000
4,frame_0040.jpg,0,0.112544,0.000000,0.112575,0.047593,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.0,0.417285,0.000000,1.408578,0.369776,0.257754,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,frame_6710.jpg,0,0.000000,1.140839,0.000000,0.000000,0.000000,0.00000,0.000000,0.488408,...,0.0,0.681457,0.0,0.079470,1.004164,0.660398,0.000000,1.037472,0.0,0.000000
672,frame_6720.jpg,26,0.088627,0.000000,0.000000,0.000000,0.000000,0.31582,0.000000,0.000000,...,0.0,0.396692,0.0,0.000000,1.125702,1.414762,0.450016,1.920763,0.0,0.000000
673,frame_6730.jpg,0,0.061834,0.000000,0.000000,0.432170,0.333206,0.00000,0.000000,0.310342,...,0.0,0.071686,0.0,0.005183,0.022516,0.805056,0.349467,1.731329,0.0,0.000000
674,frame_6740.jpg,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.067836,0.000000,...,0.0,0.288174,0.0,0.000000,0.000000,0.000000,0.000000,0.534649,0.0,0.000000
